In [1]:
from ImageData import ScanAssets, PipelineManager, TargetCompressedByType, EncodeToFileList, ConvertFormat, NIQE

import os.path
import sys

compressToFormats = [ "h264.mpeg" ]
compressToKSizes = [480] # [30, 60, 120, 240, 480]

imagesPath = os.path.join("..", "images")
compressedPath = os.path.join("..", "compressed")

statistics = open(os.path.join("..", "graphs", "niqe_video_score.txt"), "w")
statistics.write(';'.join(["FilePath", "ImageFormat" "NIQEScore", "Compressed", "Size", "MaxSize"])+"\n")

for toFormat in compressToFormats:
    for toKSize in compressToKSizes:
        print("NIQE Score for compressed "+str(toFormat)+" image with size of "+str(toKSize)+"K:")
        f = ScanAssets(imagesPath, recursiveSearch = True)
        f.do(None)
        for imageClass in f.indexOfAssets:
            f.data = f.indexOfAssets[imageClass]
            print("Class "+imageClass+" with number of images: "+str(len(f.data)))

            m1 = PipelineManager()
            m1.addPipeline(TargetCompressedByType("jpg", sys.maxsize, path=compressedPath, preserveCompressedOnly=False))
            m1.do(f, multiCoreOverload = 0.00000000001)

            fl = EncodeToFileList(inFormat = toFormat, path=os.path.join(compressedPath, imageClass+".txt"))
            fl.do(m1)

            print("Probing format size to maximum of "+str(toKSize)+"KB ...")
            enc = TargetCompressedByType(toFormat, toKSize, path=compressedPath, preserveCompressedOnly=False)
            enc.do(fl)
            print(toFormat+"'s- final size: "+str(enc.imageDataSize)+"KB")

            # STUB: Set encoded images back to their correct path and classifiedAs. Is it needed?
            print("Encoded Files returned: "+str(len(enc.data)))

            m2 = PipelineManager()
            m2.addPipeline(ConvertFormat(toMode = "L"))
            m2.addPipeline(NIQE())
            m2.do(enc, multiCoreOverload = 1.0)

            for img in m2.data:
                statistics.write(';'.join([img.imagePath, str(img.niqe_score), "1", str(img.imageDataSize), str(toKSize)])+"\n")
            statistics.flush()

NIQE Score for compressed h264.mpeg image with size of 480K:
Skipping non image file: ../images/THU-FVFDT/37/1.jpg for class THU-FVFDT
Class MMCBNU_6000 with number of images: 120
 PipelineManager -| TargetCompressedByType:  |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Probing format size to maximum of 480KB ...
Current file size: 6530.0KB
Quality ABS: 100.0 for ./../compressed/MMCBNU_6000.txt
ADDING NEGATIVE! = 50.0
Current file size: 1402.0KB
Quality ABS: 50.0 for ./../compressed/MMCBNU_6000.txt
ADDING NEGATIVE! = 25.0
Current file size: 226.0KB
Quality ABS: 25.0 for ./../compressed/MMCBNU_6000.txt
ADDING POSITIVE! = 50.0
Current file size: 1402.0KB
Quality ABS: 25.0 for ./../compressed/MMCBNU_6000.txt
ADDING NEGATIVE! = 37.5
Current file size: 816.0KB
Quality ABS: 12.5 for ./../compressed/MMCBNU_6000.txt
ADDING NEGATIVE! = 31.25
Current file size: 466.0KB
Quality ABS: 6.25 for ./../compressed/MMCBNU_6000.txt
ADDING PO

KeyboardInterrupt: 